### Goal: calculate characteristic values of tension curve and create a plot

In [24]:
import os
import json
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

The process-function calculates some metadata (returning it) and creates a plot (saving it directly)

In [25]:
def processFile(filePath:Path, outputFolder:Path):
    data = np.loadtxt(filePath, delimiter='\t')

    # calculate meta data
    maxStrain = data[:,0].max()
    strength  = data[:,1].max()
    fit = np.polyfit(data[:6,0], data[:6,1], 1)
    youngsModulus = fit[0]
    yieldIdx = np.where( np.polyval([youngsModulus, -0.2*youngsModulus], data[:,0]) >
                         data[:,1])[0][0]
    yieldStress = data[yieldIdx,1]
    meta = {'maxStrain [%]':maxStrain, 'strength [MPa]':strength, 'youngsModulus [MPa]':round(youngsModulus,4),
                'yieldStress [MPa]':yieldStress}

    # plot
    plt.axhline(strength, color='k', linestyle=':')
    plt.axvline(maxStrain, color='k', linestyle=':')
    plt.plot(data[:8,0], np.polyval(fit, data[:8,0]), 'k-', linewidth=2)                     #original fit
    plt.plot(data[:8,0], np.polyval([youngsModulus, -0.2*youngsModulus], data[:8,0]), 'r-')  #0.2% offset
    plt.plot(data[yieldIdx,0], yieldStress, 'ro')
    plt.plot(data[:,0], data[:,1],'-')
    with open(filePath, encoding='utf-8') as fIn:
        firstLine = fIn.readline()
        plt.xlabel(firstLine[1:].split(',')[0])
        plt.ylabel(firstLine[1:].split(',')[1])
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.savefig(outputFolder/(filePath.stem+'.png'), dpi=150, bbox_inches='tight')
    plt.close()
    return meta

Main code part that loops over all input files, calls the process function and saves the accumulated metadata

In [ ]:
sourceFolder = Path('.')
outputFolder = Path('.')
outputFolder.mkdir(exist_ok=True)                       #ensure that output folder exists
files = os.listdir(sourceFolder)
metaAll = {}
for file in files:
    if file.endswith('.tsv'):
      meta = processFile(sourceFolder/file, outputFolder)
      metaAll[file] = meta
json.dump(metaAll, open(outputFolder/'metaOutput.json', 'w'), indent=2)